In [11]:
# import internal files
from historymatch import emulators
from historymatch import sample
from historymatch import historymatch
from historymatch import plot


# import external modules
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

plt.rcParams.update({'font.size': 10})

np.random.seed(4)

In [12]:
def model_eqn_1(x, theta_0, theta_1, theta_2):
    return theta_0 + theta_1*(x) + theta_2*(x**2)

function1 = np.vectorize(model_eqn_1)

In [13]:
def model_eqn_2(x, theta_0, theta_1, theta_2):
    return theta_0 - theta_1*np.sin(x) + (theta_2**1)*np.cos(x)

function2 = np.vectorize(model_eqn_2)

In [14]:
# define parameter space
var_method = 0

theta_0_bound = np.array([-1, 1]).reshape(1,-1)
theta_1_bound = np.array([-1, 1]).reshape(1,-1)
theta_2_bound = np.array([-1, 1]).reshape(1,-1)

parameter_bounds = np.concatenate((theta_0_bound, theta_1_bound, theta_2_bound), axis=0)

theta_0_vals = np.linspace(parameter_bounds[0,0], parameter_bounds[0,1], 100)
theta_1_vals = np.linspace(parameter_bounds[1,0], parameter_bounds[1,1], 100)
theta_2_vals = np.linspace(parameter_bounds[2,0], parameter_bounds[2,1], 100)

theta_vals = np.concatenate((theta_0_vals.reshape(1,-1), theta_1_vals.reshape(1,-1), theta_2_vals.reshape(1,-1)), axis=0)

In [15]:
Nx = 5

def toy_model(theta_0, theta_1, theta_2):
    
    x_bound = np.array([0.1, 1])
    Nx = 5
    xvals = np.linspace(x_bound[0], x_bound[-1], Nx)

    outputs =  []
    for x in xvals:
        output = theta_0 + (theta_1**1)*(x) + theta_2*(x**2)
        outputs.append(output)
    for x in xvals:
        output = theta_0 - theta_1*np.sin(x) + (theta_2**1)*np.cos(x)
        outputs.append(output)
    return np.array(outputs)

In [16]:
# generate observational data with some uncertainty
true_parameters = [-0.1, 0.3, 0.4]

'''output_ratio = (toy_model(-1,-1,-1) - toy_model(1,1,1)) / (toy_model(-1,-1,-1) - toy_model(1,1,1))[0]

var_obs = output_ratio*0.001 # observational uncertainty variance


error_obs = np.zeros(2*Nx)
for i in range(2*Nx):
    error_obs[i] = np.random.normal(0, np.sqrt(var_obs[i]))


obs_data = toy_model(*true_parameters) + error_obs'''


var_obs = np.ones(2*Nx)*0.01 # observational uncertainty variance

error_obs = np.random.normal(0, np.sqrt(var_obs), 2*Nx)

obs_data = toy_model(*true_parameters) + error_obs


In [17]:
nwaves = 3
ndim = 3
volshape = 'hypercube'

In [18]:
importlib.reload(historymatch)

# initialise history matching class
HM = historymatch.HistoryMatch(ndim, 'GP', volshape)

In [19]:


ToyModel = historymatch.Simulator(HM)
ToyModel.set_simulator(toy_model)

HM.set_observations(obs_data, np.sqrt(var_obs))
HM.initialize_volume([-1,-1,-1], [1,1,1])

results = HM.run(nwaves=nwaves)

(3, 2)
(3, 2)
Running wave 1
(1,)
(8,)
Emulating output 0...
(1,)
(8,)
Emulating output 1...


KeyboardInterrupt: 

In [ ]:
importlib.reload(plot)

fig, axes = plt.subplots(ndim,ndim,figsize=(10,10))

plot.plotcorner(results.samples[2], parameter_bounds, 3, Fig=(fig,axes), labels=('theta1', 'theta2', 'theta3'))
axes[1,0].scatter(true_parameters[0],true_parameters[1], color='red')
axes[2,0].scatter(true_parameters[0],true_parameters[2], color='red')
axes[2,1].scatter(true_parameters[1],true_parameters[2], color='red')

In [ ]:
# save results for plotting


if volshape == 'ellipsoid':
    with open('data/3D_historymatch_ellipsoid_samples.txt', 'w') as file:
        for i in range(len(results.samples)):
            np.savetxt(file, results.samples[i])

    with open('data/3D_historymatch_ellipsoid_w1.txt', 'w') as file:
        np.savetxt(file, results.regions[0])
    with open('data/3D_historymatch_ellipsoid_w2.txt', 'w') as file:
        np.savetxt(file, results.regions[1])
    with open('data/3D_historymatch_ellipsoid_w3.txt', 'w') as file:
        np.savetxt(file, results.regions[2])
        
elif volshape == 'hypercube':

    with open('data/3D_historymatch_hypercube_samples.txt', 'w') as file:
        for i in range(len(results.samples)):
            np.savetxt(file, results.samples[i])

    with open('data/3D_historymatch_hypercube_w1.txt', 'w') as file:
        np.savetxt(file, results.regions[0])
    with open('data/3D_historymatch_hypercube_w2.txt', 'w') as file:
        np.savetxt(file, results.regions[1])
    with open('data/3D_historymatch_hypercube_w3.txt', 'w') as file:
        np.savetxt(file, results.regions[2])
        
elif volshape == 'hypercube_rot':

    with open('data/3D_historymatch_hypercube_r_samples.txt', 'w') as file:
        for i in range(len(results.samples)):
            np.savetxt(file, results.samples[i])

    with open('data/3D_historymatch_hypercube_r_w1.txt', 'w') as file:
        np.savetxt(file, results.regions[0])
    with open('data/3D_historymatch_hypercube_r_w2.txt', 'w') as file:
        np.savetxt(file, results.regions[1])
    with open('data/3D_historymatch_hypercube_r_w3.txt', 'w') as file:
        np.savetxt(file, results.nonimp_volumes[2])

#with open('data/EL_Itrain.txt', 'w') as file:
    #np.savetxt(file, results.Itrain)
'''
with open('data/HC_Itrain.txt', 'w') as file:
    for i in range(len(results.Itrain)):
        np.savetxt(file, results.Itrain[i])
        
with open('data/HC_Idata.txt', 'w') as file:
    for i in range(len(results.Idata)):
        np.savetxt(file, results.Idata[i])
'''
